In [67]:
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
import matplotlib.pyplot as plt
import sklearn.metrics as mtr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle


In [68]:
#Lendo dados de clima e convertendo a data pra datetime (na leitura veio como objeto)
clima = pd.read_csv('../Clean/Clima_FL.csv')
clima['Datetime'] = pd.to_datetime(clima['Datetime'])
clima

,Precipitacao,Datetime,Temperatura,Chuva,Calor
0,0.0,2018-12-31 22:00:00,27.15,0,2
1,0.0,2018-12-31 23:00:00,27.00,0,2
2,0.0,2019-01-01 00:00:00,26.75,0,2
3,0.0,2019-01-01 01:00:00,26.50,0,2
4,0.0,2019-01-01 02:00:00,26.45,0,2
...,...,...,...,...,...
8755,0.0,2019-12-31 16:00:00,29.40,0,2
8756,0.0,2019-12-31 17:00:00,28.55,0,2
8757,0.0,2019-12-31 18:00:00,27.65,0,2
8758,0.0,2019-12-31 19:00:00,26.70,0,2


In [69]:
rota = "rota138"
df = pd.read_csv(f"../Clean/{rota}FL.csv")
df

,data_partida,DataFimm,tempo_viagem,partidaTimeStamp,dia_semana,tipo_dia,hora_dia,turno_dia,qtdDiasAno
0,2019-04-08 19:32:00,2019-04-08 19:59:00,1620,1554751920,1,1,19,4,98
1,2019-04-30 05:12:00,2019-04-30 05:39:00,1620,1556601120,2,1,5,2,120
2,2019-04-30 05:37:00,2019-04-30 06:24:00,2820,1556602620,2,1,5,2,120
3,2019-04-30 06:02:00,2019-04-30 06:53:00,3060,1556604120,2,1,6,2,120
4,2019-04-30 06:16:00,2019-04-30 07:10:00,3240,1556604960,2,1,6,2,120
...,...,...,...,...,...,...,...,...,...
12123,2019-09-01 20:34:00,2019-09-01 21:21:00,2820,1567370040,7,1,20,4,244
12124,2019-09-01 21:19:00,2019-09-01 22:00:00,2460,1567372740,7,1,21,4,244
12125,2019-09-01 22:01:00,2019-09-01 22:45:00,2640,1567375260,7,1,22,4,244
12126,2019-09-01 22:46:00,2019-09-01 23:30:00,2640,1567377960,7,1,22,4,244


In [70]:
#copiando o DAtaframe original para não alterar o primeiro
aux = df.copy()
#Mudanças na data_partida para conseguir fazer o merge
aux["data_partida2"] = pd.to_datetime(aux["data_partida"])
aux["data_partida2"] = aux["data_partida2"].dt.round("H")
#Merge dos dados e exclusão da coluna auxiliar
df_clima = aux.merge(clima, left_on="data_partida2", right_on="Datetime", how="left")
df_clima.drop(['data_partida2','Datetime'],axis=1,inplace=True)
# df_clima.index = aux.index
df_clima

,data_partida,DataFimm,tempo_viagem,partidaTimeStamp,dia_semana,tipo_dia,hora_dia,turno_dia,qtdDiasAno,Precipitacao,Temperatura,Chuva,Calor
0,2019-04-08 19:32:00,2019-04-08 19:59:00,1620,1554751920,1,1,19,4,98,0.0,21.80,0.0,1.0
1,2019-04-30 05:12:00,2019-04-30 05:39:00,1620,1556601120,2,1,5,2,120,0.0,17.10,0.0,1.0
2,2019-04-30 05:37:00,2019-04-30 06:24:00,2820,1556602620,2,1,5,2,120,0.0,17.10,0.0,1.0
3,2019-04-30 06:02:00,2019-04-30 06:53:00,3060,1556604120,2,1,6,2,120,0.0,17.10,0.0,1.0
4,2019-04-30 06:16:00,2019-04-30 07:10:00,3240,1556604960,2,1,6,2,120,0.0,17.10,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12125,2019-09-01 20:34:00,2019-09-01 21:21:00,2820,1567370040,7,1,20,4,244,0.0,16.90,0.0,1.0
12126,2019-09-01 21:19:00,2019-09-01 22:00:00,2460,1567372740,7,1,21,4,244,0.0,16.90,0.0,1.0
12127,2019-09-01 22:01:00,2019-09-01 22:45:00,2640,1567375260,7,1,22,4,244,0.0,16.80,0.0,1.0
12128,2019-09-01 22:46:00,2019-09-01 23:30:00,2640,1567377960,7,1,22,4,244,0.0,16.85,0.0,1.0


In [71]:
df_tempos = df_clima.copy()
df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_5"] = df_tempos["tempo_viagem"]

df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem_1"].shift(1)
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem_2"].shift(2)
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem_3"].shift(3)
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem_4"].shift(4)
df_tempos["tempo_viagem_5"] = df_tempos["tempo_viagem_5"].shift(5)

df_tempos = df_tempos.iloc[5:]

df_tempos.head()

,data_partida,DataFimm,tempo_viagem,partidaTimeStamp,dia_semana,tipo_dia,hora_dia,turno_dia,qtdDiasAno,Precipitacao,Temperatura,Chuva,Calor,tempo_viagem_1,tempo_viagem_2,tempo_viagem_3,tempo_viagem_4,tempo_viagem_5
5,2019-04-30 06:33:00,2019-04-30 07:37:00,3840,1556605980,2,1,6,2,120,0.0,16.55,0.0,1.0,3240.0,3060.0,2820.0,1620.0,1620.0
6,2019-04-30 06:55:00,2019-04-30 08:07:00,4320,1556607300,2,1,6,2,120,0.0,16.55,0.0,1.0,3840.0,3240.0,3060.0,2820.0,1620.0
7,2019-04-30 07:10:00,2019-04-30 08:20:00,4200,1556608200,2,1,7,2,120,0.0,16.55,0.0,1.0,4320.0,3840.0,3240.0,3060.0,2820.0
8,2019-04-30 07:35:00,2019-04-30 08:40:00,3900,1556609700,2,1,7,2,120,0.2,16.65,1.0,1.0,4200.0,4320.0,3840.0,3240.0,3060.0
9,2019-04-30 07:50:00,2019-04-30 09:00:00,4200,1556610600,2,1,7,2,120,0.2,16.65,1.0,1.0,3900.0,4200.0,4320.0,3840.0,3240.0


In [72]:
df_tempos.to_csv(f"../Clean/{rota}FL_Full.csv")